This script will read in all the images, crop them to the face, and then calculate the mean luminance.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
from glob import glob

import numpy as np
import pandas as pd

import skimage.io as io
io.use_plugin('matplotlib')

import menpo.io as mio
from menpowidgets import visualize_images
from pathlib import Path

from tqdm import tqdm

In [2]:
unfam_df = pd.read_csv("/data1/famface01/analysis/encoding/12_Features/unfam_final_frame_df.csv")
unfam_df = unfam_df.ix[:,1:]
unfam_df.head()

,vnum,vid,frame,category,fpath
0,14,Ahmed_Chalabi_U000_vid06_clip05_2secs,3,unfam,/data1/famface01/data/stimuli/vids/eight_frame...
1,14,Ahmed_Chalabi_U000_vid06_clip05_2secs,9,unfam,/data1/famface01/data/stimuli/vids/eight_frame...
2,14,Ahmed_Chalabi_U000_vid06_clip05_2secs,15,unfam,/data1/famface01/data/stimuli/vids/eight_frame...
3,14,Ahmed_Chalabi_U000_vid06_clip05_2secs,21,unfam,/data1/famface01/data/stimuli/vids/eight_frame...
4,14,Ahmed_Chalabi_U000_vid06_clip05_2secs,27,unfam,/data1/famface01/data/stimuli/vids/eight_frame...


Load the images and crop to the face.

In [3]:
from menpo.landmark import face_ibug_68_to_face_ibug_68_trimesh, face_ibug_68_to_face_ibug_68, labeller

unfam_images = []
for i,row in tqdm(unfam_df.iterrows(), total=len(unfam_df)):
    img = mio.import_image(row.fpath)
    img = img.crop_to_landmarks_proportion(0.1)
    labeller(img, 'PTS', face_ibug_68_to_face_ibug_68_trimesh)
    unfam_images.append(img)

100%|██████████| 6912/6912 [03:28<00:00, 35.00it/s]


Calculate the average luminance for each image.

In [5]:
mean_lumis = np.array([ img.as_greyscale().as_vector().mean() for img in tqdm(unfam_images) ])

100%|██████████| 6912/6912 [00:02<00:00, 3245.11it/s]


In [15]:
odf = pd.DataFrame({'vid': unfam_df.vid, 'frame': unfam_df.frame, 'luminance': mean_lumis}, columns=['vid', 'frame', 'luminance'])
odf.head()

,vid,frame,luminance
0,Ahmed_Chalabi_U000_vid06_clip05_2secs,3,0.401390
1,Ahmed_Chalabi_U000_vid06_clip05_2secs,9,0.398201
2,Ahmed_Chalabi_U000_vid06_clip05_2secs,15,0.404308
3,Ahmed_Chalabi_U000_vid06_clip05_2secs,21,0.402420
4,Ahmed_Chalabi_U000_vid06_clip05_2secs,27,0.399624


In [16]:
odf.to_csv('/data1/famface01/analysis/misc/400_nnet_specificity/feats_ave_luminance.csv')